In [49]:
import pandas as pd
import re

In [50]:
df = pd.read_csv('real_estate_listings.csv')
# cdf = df
df

,Location,Price,Type of House,Land Area,Bedrooms,Toilets,Total Floors,Main Door Direction,Balcony Direction,Legal Documents
0,"Phường 15, Quận Bình Thạnh","4,16 tỷ","Nhà hẻm, ngõ","36 m² (3,2x12,0)",4 phòng,3 WC,2.0,NaN,NaN,Sổ hồng
1,"Xã Tân Kiên, Huyện Bình Chánh",1 tỷ,Nhà mặt tiền,"93,9 m² (5,0x18,8)",3 phòng,2 WC,1.0,NaN,NaN,Sổ hồng
2,"Phường Thạnh Xuân, Quận 12","6,5 tỷ","Nhà hẻm, ngõ","138 m² (8,2x17,0)",3 phòng,3 WC,2.0,NaN,NaN,Sổ hồng
3,"Phường 5, Quận Gò Vấp",23 tỷ,"Nhà hẻm, ngõ","135 m² (5,0x27,0)",21 phòng,21 WC,5.0,Tây Nam,NaN,Sổ hồng
4,"Phường 2, Quận Tân Bình",NaN,"Biệt thự, Villa","100 m² (5,0x20,0)",5 phòng,NaN,NaN,NaN,NaN,Sổ hồng
...,...,...,...,...,...,...,...,...,...,...
26043,"Xã Thới Tam Thôn, Huyện Hóc Môn",4 tỷ,Đất thổ cư,"88 m² (5,0x17,5)",NaN,NaN,NaN,NaN,NaN,Sổ hồng
26044,"Phường 15, Quận Tân Bình","7,85 tỷ","Nhà hẻm, ngõ","64 m² (4,0x16,0)",4 phòng,3 WC,NaN,NaN,NaN,Sổ hồng
26045,"Phường 14, Quận Gò Vấp","4,66 tỷ","Nhà hẻm, ngõ","42 m² (3,5x12,0)",2 phòng,2 WC,1.0,NaN,NaN,Sổ hồng
26046,"Xã Tân Hiệp, Huyện Hóc Môn","3,45 tỷ",Đất thổ cư,"120 m² (7,4x16,0)",NaN,NaN,NaN,Tây Nam,NaN,Sổ hồng


In [51]:
nan_counts = df.isna().sum()
nan_counts

Location                   0
Price                   1022
Type of House              0
Land Area                 55
Bedrooms               12991
Toilets                15243
Total Floors           20037
Main Door Direction    21409
Balcony Direction      23452
Legal Documents            0
dtype: int64

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26048 entries, 0 to 26047
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Location             26048 non-null  object 
 1   Price                25026 non-null  object 
 2   Type of House        26048 non-null  object 
 3   Land Area            25993 non-null  object 
 4   Bedrooms             13057 non-null  object 
 5   Toilets              10805 non-null  object 
 6   Total Floors         6011 non-null   float64
 7   Main Door Direction  4639 non-null   object 
 8   Balcony Direction    2596 non-null   object 
 9   Legal Documents      26048 non-null  object 
dtypes: float64(1), object(9)
memory usage: 2.0+ MB


In [53]:
df.drop(columns=["Main Door Direction", "Balcony Direction", "Legal Documents"], inplace=True)

# Loại bỏ các hàng thiếu dữ liệu ở cột Price hoặc Land Area
df = df.dropna(subset=["Price", "Land Area"])

In [54]:
nan_counts = df.isna().sum()
nan_counts

Location             0
Price                0
Type of House        0
Land Area            0
Bedrooms         12928
Toilets          15175
Total Floors     19006
dtype: int64

In [55]:
def chuan_hoa_gia_ve_vnd(gia_str):
    gia_str = str(gia_str).replace('đ', '').replace('.', '').strip().lower()

    ty = 0
    trieu = 0

    # Trường hợp "x tỷ y triệu"
    match = re.match(r'([\d,\.]+)\s*tỷ\s*([\d,\.]*)\s*triệu*', gia_str)
    if match:
        ty = float(match.group(1).replace(',', '.'))
        trieu = float(match.group(2).replace(',', '.')) if match.group(2) else 0

    elif 'tỷ' in gia_str:
        ty = float(gia_str.replace('tỷ', '').replace(',', '.').strip())

    elif 'triệu' in gia_str:
        trieu = float(gia_str.replace('triệu', '').replace(',', '.').strip())

    else:
        # Trường hợp là số trực tiếp như "650000000"
        try:
            return int(re.sub(r'[^\d]', '', gia_str))
        except:
            return np.nan

    tong_vnd = int((ty * 1_000 + trieu) * 1_000_000)
    return tong_vnd

# Áp dụng vào DataFrame
df['Price'] = df['Price'].apply(chuan_hoa_gia_ve_vnd)

df["Bedrooms"] = df["Bedrooms"].astype(str).str.extract(r"(\d+)").astype(float)
df["Toilets"] = df["Toilets"].astype(str).str.extract(r"(\d+)").astype(float)
df['Land Area'] = df['Land Area'].astype(str).str.extract(r"(\d+)").astype(float)

df

C:\Users\maiho\AppData\Local\Temp\ipykernel_872\478173610.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Price'] = df['Price'].apply(chuan_hoa_gia_ve_vnd)
C:\Users\maiho\AppData\Local\Temp\ipykernel_872\478173610.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Bedrooms"] = df["Bedrooms"].astype(str).str.extract(r"(\d+)").astype(float)
C:\Users\maiho\AppData\Local\Temp\ipykernel_872\478173610.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

,Location,Price,Type of House,Land Area,Bedrooms,Toilets,Total Floors
0,"Phường 15, Quận Bình Thạnh",4160000000,"Nhà hẻm, ngõ",36.0,4.0,3.0,2.0
1,"Xã Tân Kiên, Huyện Bình Chánh",1000000000,Nhà mặt tiền,93.0,3.0,2.0,1.0
2,"Phường Thạnh Xuân, Quận 12",6500000000,"Nhà hẻm, ngõ",138.0,3.0,3.0,2.0
3,"Phường 5, Quận Gò Vấp",23000000000,"Nhà hẻm, ngõ",135.0,21.0,21.0,5.0
5,"Xã Phạm Văn Hai, Huyện Bình Chánh",38000000000,Nhà mặt tiền,900.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
26043,"Xã Thới Tam Thôn, Huyện Hóc Môn",4000000000,Đất thổ cư,88.0,NaN,NaN,NaN
26044,"Phường 15, Quận Tân Bình",7850000000,"Nhà hẻm, ngõ",64.0,4.0,3.0,NaN
26045,"Phường 14, Quận Gò Vấp",4660000000,"Nhà hẻm, ngõ",42.0,2.0,2.0,1.0
26046,"Xã Tân Hiệp, Huyện Hóc Môn",3450000000,Đất thổ cư,120.0,NaN,NaN,NaN


In [56]:
df0 = df
df1 = df0.dropna(subset=["Bedrooms"])
df1.drop(columns=["Toilets", "Total Floors"], inplace=True)
df1

C:\Users\maiho\AppData\Local\Temp\ipykernel_872\1182539553.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.drop(columns=["Toilets", "Total Floors"], inplace=True)


,Location,Price,Type of House,Land Area,Bedrooms
0,"Phường 15, Quận Bình Thạnh",4160000000,"Nhà hẻm, ngõ",36.0,4.0
1,"Xã Tân Kiên, Huyện Bình Chánh",1000000000,Nhà mặt tiền,93.0,3.0
2,"Phường Thạnh Xuân, Quận 12",6500000000,"Nhà hẻm, ngõ",138.0,3.0
3,"Phường 5, Quận Gò Vấp",23000000000,"Nhà hẻm, ngõ",135.0,21.0
7,"Phường 14, Quận 3",43500000000,"Biệt thự, Villa",160.0,9.0
...,...,...,...,...,...
26040,"Phường 13, Quận 6",25000000000,Nhà mặt tiền,125.0,7.0
26041,"Phường Bình Trị Đông B, Quận Bình Tân",10500000000,Nhà mặt tiền,80.0,5.0
26042,"Xã Thới Tam Thôn, Huyện Hóc Môn",4900000000,Nhà mặt tiền,90.0,2.0
26044,"Phường 15, Quận Tân Bình",7850000000,"Nhà hẻm, ngõ",64.0,4.0


In [57]:
df1.to_csv("df1.csv", index=False, encoding='utf-8-sig')

In [58]:
df['Bedrooms'] = df.apply(lambda row: row['Toilets'] if pd.isna(row['Bedrooms']) and pd.notna(row['Toilets']) else row['Bedrooms'], axis=1)

df['Bedrooms'] = df.apply(
    lambda row: (row['Land Area'] / 20) * (row['Total Floors'] - 1) 
    if pd.isna(row['Bedrooms']) and pd.notna(row['Total Floors']) and pd.notna(row['Land Area']) 
    else row['Bedrooms'],
    axis=1
)

df

C:\Users\maiho\AppData\Local\Temp\ipykernel_872\2794205200.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Bedrooms'] = df.apply(lambda row: row['Toilets'] if pd.isna(row['Bedrooms']) and pd.notna(row['Toilets']) else row['Bedrooms'], axis=1)
C:\Users\maiho\AppData\Local\Temp\ipykernel_872\2794205200.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Bedrooms'] = df.apply(


,Location,Price,Type of House,Land Area,Bedrooms,Toilets,Total Floors
0,"Phường 15, Quận Bình Thạnh",4160000000,"Nhà hẻm, ngõ",36.0,4.0,3.0,2.0
1,"Xã Tân Kiên, Huyện Bình Chánh",1000000000,Nhà mặt tiền,93.0,3.0,2.0,1.0
2,"Phường Thạnh Xuân, Quận 12",6500000000,"Nhà hẻm, ngõ",138.0,3.0,3.0,2.0
3,"Phường 5, Quận Gò Vấp",23000000000,"Nhà hẻm, ngõ",135.0,21.0,21.0,5.0
5,"Xã Phạm Văn Hai, Huyện Bình Chánh",38000000000,Nhà mặt tiền,900.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
26043,"Xã Thới Tam Thôn, Huyện Hóc Môn",4000000000,Đất thổ cư,88.0,NaN,NaN,NaN
26044,"Phường 15, Quận Tân Bình",7850000000,"Nhà hẻm, ngõ",64.0,4.0,3.0,NaN
26045,"Phường 14, Quận Gò Vấp",4660000000,"Nhà hẻm, ngõ",42.0,2.0,2.0,1.0
26046,"Xã Tân Hiệp, Huyện Hóc Môn",3450000000,Đất thổ cư,120.0,NaN,NaN,NaN


In [59]:
nan_counts = df.isna().sum()
nan_counts

Location             0
Price                0
Type of House        0
Land Area            0
Bedrooms         10708
Toilets          15175
Total Floors     19006
dtype: int64

In [60]:
df2 = df.dropna(subset=["Bedrooms"])
df2.drop(columns=["Toilets", "Total Floors"], inplace=True)
df2

C:\Users\maiho\AppData\Local\Temp\ipykernel_872\3379221376.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.drop(columns=["Toilets", "Total Floors"], inplace=True)


,Location,Price,Type of House,Land Area,Bedrooms
0,"Phường 15, Quận Bình Thạnh",4160000000,"Nhà hẻm, ngõ",36.0,4.0
1,"Xã Tân Kiên, Huyện Bình Chánh",1000000000,Nhà mặt tiền,93.0,3.0
2,"Phường Thạnh Xuân, Quận 12",6500000000,"Nhà hẻm, ngõ",138.0,3.0
3,"Phường 5, Quận Gò Vấp",23000000000,"Nhà hẻm, ngõ",135.0,21.0
7,"Phường 14, Quận 3",43500000000,"Biệt thự, Villa",160.0,9.0
...,...,...,...,...,...
26040,"Phường 13, Quận 6",25000000000,Nhà mặt tiền,125.0,7.0
26041,"Phường Bình Trị Đông B, Quận Bình Tân",10500000000,Nhà mặt tiền,80.0,5.0
26042,"Xã Thới Tam Thôn, Huyện Hóc Môn",4900000000,Nhà mặt tiền,90.0,2.0
26044,"Phường 15, Quận Tân Bình",7850000000,"Nhà hẻm, ngõ",64.0,4.0


In [61]:
df2.to_csv("df2.csv", index=False, encoding='utf-8-sig')